In [2]:
using Plots
using DataFrames
import CSV
using MLJ
using LossFunctions
import FileIO

In [12]:
x_train, x_test, y = FileIO.load("Preprocessing/preprocessed.jld2", "x_train", "x_test", "y");


In [13]:
models(matching(x_train, y))

7-element Vector{NamedTuple{(:name, :package_name, :is_supervised, :docstring, :hyperparameter_ranges, :hyperparameter_types, :hyperparameters, :implemented_methods, :is_pure_julia, :is_wrapper, :iteration_parameter, :load_path, :package_license, :package_url, :package_uuid, :prediction_type, :supports_class_weights, :supports_online, :supports_training_losses, :supports_weights, :input_scitype, :target_scitype, :output_scitype), T} where T<:Tuple}:
 (name = ConstantRegressor, package_name = MLJModels, ... )
 (name = DecisionTreeRegressor, package_name = BetaML, ... )
 (name = DecisionTreeRegressor, package_name = DecisionTree, ... )
 (name = DeterministicConstantRegressor, package_name = MLJModels, ... )
 (name = RandomForestRegressor, package_name = BetaML, ... )
 (name = RandomForestRegressor, package_name = DecisionTree, ... )
 (name = RandomForestRegressor, package_name = ScikitLearn, ... )

In [14]:
@load DecisionTreeRegressor pkg="DecisionTree"
regressor = MLJDecisionTreeInterface.DecisionTreeRegressor()

import MLJDecisionTreeInterface ✔


┌ Info: For silent loading, specify `verbosity=0`. 
└ @ Main /Users/alex/.julia/packages/MLJModels/E8BbE/src/loading.jl:168


DecisionTreeRegressor(
    max_depth = -1,
    min_samples_leaf = 5,
    min_samples_split = 2,
    min_purity_increase = 0.0,
    n_subfeatures = 0,
    post_prune = false,
    merge_purity_threshold = 1.0) @917

In [15]:
pipe = @pipeline(regressor )

Pipeline258(
    decision_tree_regressor = DecisionTreeRegressor(
            max_depth = -1,
            min_samples_leaf = 5,
            min_samples_split = 2,
            min_purity_increase = 0.0,
            n_subfeatures = 0,
            post_prune = false,
            merge_purity_threshold = 1.0)) @472

In [17]:
regress = machine(pipe, x_train, y)

Machine{Pipeline258,…} @341 trained 0 times; caches data
  args: 
    1:	Source @168 ⏎ `Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}`
    2:	Source @987 ⏎ `AbstractVector{Continuous}`


In [18]:
train, val = partition(eachindex(y), 0.7, shuffle=true, rng=0)

([47089, 51374, 88757, 271742, 43265, 189152, 328550, 90194, 240676, 319288  …  1369, 350910, 410195, 74497, 224591, 43365, 280402, 90253, 351563, 77052], [247422, 414679, 148270, 79143, 135363, 149249, 301974, 90332, 234061, 321352  …  283929, 155360, 229652, 256257, 306923, 86417, 220214, 25498, 98387, 204353])

In [19]:
fit!(regress, rows=train)

┌ Info: Training Machine{Pipeline258,…} @341.
└ @ MLJBase /Users/alex/.julia/packages/MLJBase/hLtde/src/machines.jl:342
┌ Info: Training Machine{DecisionTreeRegressor,…} @482.
└ @ MLJBase /Users/alex/.julia/packages/MLJBase/hLtde/src/machines.jl:342


Machine{Pipeline258,…} @341 trained 1 time; caches data
  args: 
    1:	Source @168 ⏎ `Table{Union{AbstractVector{Continuous}, AbstractVector{Count}}}`
    2:	Source @987 ⏎ `AbstractVector{Continuous}`


In [20]:
fitted_params(regress) |> println

(decision_tree_regressor = (tree = Decision Tree
Leaves: 47380
Depth:  38,), machines = Machine[Machine{DecisionTreeRegressor,…} @482], fitted_params_given_machine = OrderedCollections.LittleDict{Any, Any, Vector{Any}, Vector{Any}}(Machine{DecisionTreeRegressor,…} @482 => (tree = Decision Tree
Leaves: 47380
Depth:  38,)))


In [21]:
ypred = predict(regress, rows=val)


126471-element Vector{Float64}:
  24341.93625
   3275.79375
  24770.899999999998
  16989.894
  11414.494
  73199.98666666666
  32111.312857142857
  10128.192
    232.11428571428573
   3667.626
   8879.034444444444
  12572.53375
   2143.7655555555557
      ⋮
   2353.3433333333332
  13238.38
   4937.414
  10265.99
   6730.307142857144
  59613.38999999999
  29730.73
  10906.37
  11597.615555555556
 112959.898
  29608.528000000002
    439.42499999999995

In [24]:
y[val]

126471-element Vector{Float64}:
  24141.66
   3236.37
  21521.85
  17026.78
  11473.65
  90076.18
  32146.94
   8072.86
    134.55
   1479.92
  10389.96
  12531.61
   2184.95
      ⋮
   4629.0
  12386.84
   4749.0
   9820.68
   6611.54
  76107.15
  44748.01
   9780.82
  16553.22
 113768.51
  26524.66
    458.09

In [25]:
LossFunctions.value(L2DistLoss(), y[val], ypred)

126471-element Vector{Float64}:
  40110.57631406209
   1554.2320640624944
      1.0556325902499996e7
   1360.576995999897
   3499.432335999886
      2.848059014240443e8
   1269.2933081631854
      4.224389630223998e6
   9518.789846938776
      4.786057542436001e6
      2.2828960344308643e6
   1674.7533140624942
   1696.1584641975046
      ⋮
      5.178613264544445e6
 725120.3715999984
  35499.83539599991
 198300.99609999955
  14105.634222449224
      2.720441189376001e8
      2.255186985984001e8
      1.2668628025000026e6
      2.4558015409797538e7
 653853.3665439899
      9.510241841424014e6
    348.38222500000074